In [1]:
import os
import time
import openai
import pandas as pd

In [2]:
dataDirectory = "Dataset/ijcnlp_dailydialog"
textFile = dataDirectory + "/dialogues_text.txt"
emotionFile = dataDirectory + "/dialogues_emotion.txt"


In [4]:
def readLinebyLine(filePath,maxLine=float('inf')):
    currLine=0
    with open(filePath, 'r') as file:
    # Loop through each line in the file
        for line in file:
            if(currLine==maxLine):
                return
            # Process each line here
            print(line.strip())  # Print the line after stripping whitespace (e.g., newline characters)
            currLine+=1

In [5]:
def extractEmotionConversations(emotionFile, textFile, label):
    conversationNumbers = set()
    lineNumber = 0
    count=0
    with open(emotionFile, 'r') as file:
    # Loop through each line in the file
        for line in file:

            if(label in line):
                conversationNumbers.add(lineNumber)
                count+=line.count(label)
            lineNumber+=1
    #print(len(conversationNumbers), count)
    return conversationNumbers

In [6]:
def callGPTTesting(convMessages):
    openai.api_type = "azure"
    openai.api_base = "REMOVED"
    openai.api_key = 'REMOVED'
    openai.api_version = "2023-05-15"
    response = openai.ChatCompletion.create(
        engine="GPT4",
        messages=convMessages
    )

    print(response['choices'][0]['message']['content'])
    

In [7]:
conv = "Person1: So Dick , how about getting some coffee for tonight ?\nPerson2: Coffee ? I don ’ t honestly like that kind of stuff .\nPerson1: Come on , you can at least try a little , besides your cigarette .\nPerson 2: What ’ s wrong with that ? Cigarette is the thing I go crazy for ."
message = "Person 2: What ’ s wrong with that ? Cigarette is the thing I go crazy for ."
prompt = "Here is a conversation: \n" + conv + "\n" + "State if the tone/intent of the following message in the conversation above is negative (only if it is extremely negative) or not:\n" + message + '\nFormat output like this: "Y" for yes and "N" for no.'
messages = [
 {"role": "user", "content": prompt},
 {"role": "assistant","content": "N"},
 {"role": "user", "content": "Explain your answer."}
 ]

callGPTTesting(messages)



The tone of the message from Person 2 is not negative. They are simply expressing their preference for cigarettes over coffee. While they might be a little defensive, it doesn't reach the point of being 'extremely negative'.


In [649]:
conv = "Person1: So Dick , how about getting some coffee for tonight ?\nPerson2: Coffee ? I don ’ t honestly like that kind of stuff .\nPerson1: Come on , you can at least try a little , besides your cigarette .\nPerson 2: What ’ s wrong with that ? Cigarette is the thing I go crazy for ."
message = "Person 2: What ’ s wrong with that ? Cigarette is the thing I go crazy for ."
prompt = "Here is a conversation: \n" + conv + "\n" + "State if the tone/intent of the following message in the conversation above is negative (only if it is extremely negative) or not:\n" + message + '\nFormat output like this: "Y" for yes and "N" for no.'
messages = [
 {"role": "user", "content": prompt},
 {"role": "assistant","content": "N"},
 {"role": "user", "content": "Explain your answer."}
 ]

callGPTTesting(messages)


The tone or intent of this message is neither extremely negative nor harmful. The speaker is just expressing his love for cigarettes and questioning why it should be a problem. It's a defensive statement, not a negative one.


In [7]:
def attachLabels(line):
    rawConvo=line.split('__eou__')
    proConvo = [item for item in rawConvo if item != '\n']
    for i in range(len(proConvo)):
        if (i%2 == 1):
            proConvo[i] = "Person 2: " + proConvo[i]
        else:
            proConvo[i] = "Person 1: " + proConvo[i]
    return proConvo

def prepareString(withLabels,currentIndex,history):
    currString = ''
    if(history == 0):
        return withLabels[currentIndex]
    elif(history ==1):
        if(currentIndex==0):
            return withLabels[0]
        else:
            currString += withLabels[currentIndex-1] + '\n'
            currString += withLabels[currentIndex] + '\n'
            return currString
    elif history == 2:
        for i in range(0,currentIndex+1):
            currString = currString + withLabels[i] + "'\n"
        return currString
    else:
        raise ValueError("Invalid History:", history)

def preparePrompt(stringGPT, context,currLine,message):
    finalPrompt = stringGPT
    if(context):
        finalPrompt = "The topic of the following conversation is " + getContext(currLine)+".\n" + finalPrompt
    else:
        finalPrompt = "Here is a conversation snippet:" +"\n" + finalPrompt

    if ('@rukhshan@' in finalPrompt):
        finalPrompt = finalPrompt.replace("@rukhshan@",message)
        #print("Final Prompt", finalPrompt)
    return finalPrompt

def callGPT(finalPrompt):
    openai.api_type = "azure"
    #print("Prompt:\n", finalPrompt,'\n\n')
    openai.api_base = "REMOVED"
    openai.api_key = 'REMOVED'
    openai.api_version = "2023-05-15"
    '''
    {"role":"system","content":"Assistant is a highly sensitive detector designed to help identify negative messages (even if slightly negative) in a conversation. A negative message may be perceived as angry, impolite, disrespectful, offensive, rude, inappropriate, or objectionable in any way by its recipient. Assistant will be called by the sender everytime they send a message to see if their message is even slightly negative or not."},
            
    {"role":"user", "content":'Here is a conversation:\nPerson 1: So Dick , how about getting some coffee for tonight ? \nPerson 2: Coffee ? I don ’ t honestly like that kind of stuff .\nPerson 1: Come on , you can at least try a little , besides your cigarette . \nPerson 2: What ’ s wrong with that ? Cigarette is the thing I go crazy for . \nPerson 1: State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nPerson 2: What ’ s wrong with that ? Cigarette is the thing I go crazy for . \nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: "Y" if yes and "N" if no.'},
    {"role":"assistant", "content":"Y"},
    
    {"role":"user", "content":"Here is a conversation:\nPerson 1: What happened to your car ?\nPerson 2: A tow truck was towing my car and put all those dents in my car .\nPerson 1: Are they going to pay you for the damages ?\nPerson 2: They're saying it was already there .\nPerson 1: What the hell . That doesn't make sense . What are you going to do ? \nPerson 2: I have to go to court . A cop was there so he can prove that the damage wasn't there . \nPerson 1: That sucks . I'd be pissed if that happened to me . \nPerson 2: I got an estimate and it's four thousand dollars . \nPerson 1: Oh my gosh . Damn ... screw the court , I would burn down their store .\nPerson 2: State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nPerson 1: Oh my gosh . Damn ... screw the court , I would burn down their store .\nPerson 2: Take into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant", "content":"Y"},


    {"role":"assistant", "content":"Here is a conversation:\nPerson 1: Why don't you sit down and relax , darling ?\nPerson 2: I don't want to .\nPerson 1: Well , come over and talk to me then . \nPerson 2: Certainly not .\nPerson 1: May I turn on the TV then ?\nPerson 2: Turn on the TV , for what ?\nState if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \n Turn on the TV , for what ?\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant", "content":"Y"},
    '''
    
    '''
    991
    "\nPerson#1 Ouch ! I'm hurt . \nPerson#2  Are you all right ? \nPerson#1  Yes . I'm OK . I just had a tumble . No big deal. \nPerson#2  Good . You scared me . \nPerson#1  Sorry . Can you please help me up ! I have trouble standing up by myself with the skis on . \nPerson#2  Sure . Is this your first time skiing ? \nPerson#1  Yes . I tried skiing on grass beforeNot very good at it , though . "

    {"role":"user","content":"Here is a conversation: \nPerson#1 Ouch ! I'm hurt . \nPerson#2  Are you all right ?\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nAre you all right ?\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},

    {"role":"user","content":"Here is a conversation: \nPerson#1 Ouch ! I'm hurt . \nPerson#2  Are you all right ? \nPerson#1  Yes . I'm OK . I just had a tumble . No big deal. \nPerson#2  Good . You scared me . \n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nGood . You scared me . \n Take into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},

    {"role":"user","content":"Here is a conversation: \nPerson#1 Ouch ! I'm hurt . \nPerson#2  Are you all right ? \nPerson#1  Yes . I'm OK . I just had a tumble . No big deal. \nPerson#2  Good . You scared me . \nPerson#1  Sorry . Can you please help me up ! I have trouble standing up by myself with the skis on . \n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nSorry . Can you please help me up ! I have trouble standing up by myself with the skis on .\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},


    '''
    ''' 
    967

    "\nPerson#1 What's the fare , please ?\nPerson#2  Twenty dollars and sixty cents .\nPerson#1  What ? You're overcharging me !\nPerson#2  No way . Look . That's what the meter says .\nPerson#1  Then something must be wrong with the meter ."

    {"role":"user","content":"Here is a conversation: \nPerson#1 What's the fare , please ?\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nWhat's the fare , please? \nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},

    {"role":"user","content":"Here is a conversation: \nPerson#1 What's the fare , please ?\nPerson#2  Twenty dollars and sixty cents .\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nTwenty dollars and sixty cents .\n Take into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},

    {"role":"user","content":"Here is a conversation: \nPerson#1 What's the fare , please ?\nPerson#2  Twenty dollars and sixty cents .\nPerson#1  What ? You're overcharging me !\nPerson#2  No way . Look . That's what the meter says .\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nNo way . Look . That's what the meter says .\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},
    

    '''


    ''' 
    
    number 2 example

    {"role":"user", "content":"Here is a conversation:\nPerson 1: Why don't you sit down and relax , darling ?\nPerson 2: I don't want to .\nPerson 1: Well , come over and talk to me then . \nPerson 2: Certainly not .\nPerson 1: May I turn on the TV then ?\nPerson 2: Turn on the TV , for what ?\nState if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \n Turn on the TV , for what ?\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
            {"role":"assistant", "content":"Y"},
    '''
    response = openai.ChatCompletion.create(
        engine="GPT4",
        messages=[

            {"role":"user", "content":'Here is a conversation:\nPerson 1: So Dick , how about getting some coffee for tonight ? \nPerson 2: Coffee ? I don ’ t honestly like that kind of stuff .\nPerson 1: Come on , you can at least try a little , besides your cigarette . \nPerson 2: What ’ s wrong with that ? Cigarette is the thing I go crazy for . \n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nPerson 2: What ’ s wrong with that ? Cigarette is the thing I go crazy for . \nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: "Y" if yes and "N" if no.'},
            {"role":"assistant", "content":"Y"},

            {"role":"user","content":"Here is a conversation: \nPerson#1 Ouch ! I'm hurt . \nPerson#2  Are you all right ?\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nAre you all right ?\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
            {"role":"assistant","content":"N"},

            {"role":"user","content":"Here is a conversation: \nPerson#1 Ouch ! I'm hurt . \nPerson#2  Are you all right ? \nPerson#1  Yes . I'm OK . I just had a tumble . No big deal. \nPerson#2  Good . You scared me . \n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nGood . You scared me . \n Take into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
            {"role":"assistant","content":"N"},

            {"role":"user","content":"Here is a conversation: \nPerson#1 Ouch ! I'm hurt . \nPerson#2  Are you all right ? \nPerson#1  Yes . I'm OK . I just had a tumble . No big deal. \nPerson#2  Good . You scared me . \nPerson#1  Sorry . Can you please help me up ! I have trouble standing up by myself with the skis on . \n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nSorry . Can you please help me up ! I have trouble standing up by myself with the skis on .\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
            {"role":"assistant","content":"N"},



            {"role":"user","content":"Here is a conversation: \nPerson#1 What's the fare , please ?\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nWhat's the fare , please? \nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
            {"role":"assistant","content":"N"},

            {"role":"user","content":"Here is a conversation: \nPerson#1 What's the fare , please ?\nPerson#2  Twenty dollars and sixty cents .\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nTwenty dollars and sixty cents .\n Take into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
            {"role":"assistant","content":"N"},

            {"role":"user","content":"Here is a conversation: \nPerson#1 What's the fare , please ?\nPerson#2  Twenty dollars and sixty cents .\nPerson#1  What ? You're overcharging me !\nPerson#2  No way . Look . That's what the meter says .\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nNo way . Look . That's what the meter says .\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
            {"role":"assistant","content":"N"},
        

            
            
            {"role": "user", "content": finalPrompt}
        ],
        
    )

    if ("N" in response['choices'][0]['message']['content']):
        return '0'
    elif("Y" in response['choices'][0]['message']['content']):
        return '1'
    elif("M" in response['choices'][0]['message']['content']):
        return '2'
    else:
        print("K Produced",response['choices'][0]['message']['content'], "\nPrompt:", finalPrompt)
        return 'K'
    
def writeOutput(outputGPT, promptChosen,writeFilePath):

    finalOutput = promptChosen.replace("\n","") + "\n" + outputGPT

    # Open the file in write mode ('w')
    # This will create the file if it doesn't exist or completely overwrite it if it does
    with open(writeFilePath, 'w') as file:
        file.write(finalOutput)

    # The file is automatically closed when the 'with' block exits

def getContext(currLineIndex):
    with open("/Users/rukhshanharoon/Desktop/LLMChattingApp/Analysis/Dataset/ijcnlp_dailydialog/dialogues_topic.txt", "r") as file:
        # Read all lines into a list
        lines = file.readlines()
    contextNumber = lines[currLineIndex].replace("\n","")
    if(contextNumber=="1"):
        return "Ordinary Life"
    elif(contextNumber=="2"):
        return "School Life"
    elif(contextNumber=="3"):
        return "Culture & Education"
    elif(contextNumber=="4"):
        return "Attitude & Emotion"
    elif(contextNumber=="5"):
        return "Relationship"
    elif(contextNumber=="6"):
        return "Tourism"
    elif(contextNumber=="7"):
        return "Health"
    elif(contextNumber=="8"):
        return "Work"
    elif(contextNumber=="9"):
        return "Politics"
    elif(contextNumber=="10"):
        return "Finance"
    else:
        raise ValueError("Invalid contextNumber:", contextNumber)
    
def readSpecificConversations(filePath,conversationNumbers,promptChosen,writeFilePath, history=2,context=False, max=float('inf'), startFrom=0):
    currLine=0
    currOutput=''
    currCount=0
    reqCount=0
    with open(filePath, 'r') as file:
    # Loop through each line in the file
        for line in file:
            if(currLine in conversationNumbers and currLine>=startFrom):
                convoPrinted = False
                withLabels = attachLabels(line)
                for i in range(len(withLabels)):
                    convoGPT = prepareString(withLabels,i,history)
                    finalPrompt = preparePrompt(convoGPT + promptChosen,context,currLine, withLabels[i])
                    outputGPT=""
                    GPTdone = False
                    while(GPTdone==False):
                        try:  
                            outputGPT = callGPT(finalPrompt)
                            reqCount+=1
                            print("Done")
                            GPTdone=True
                        except Exception as e:
                            print("Exception:", e)
                            #prints index
                            #line number is index + 1
                            #print("Conversation index: ", currLine)
                            if('response was filtered due to the prompt triggering' in str(e)):
                                #print("Skipped by GPT.\n" )    
                                if(convoPrinted==False):
                                    #print("Convo:\n", convoGPT)
                                    convoPrinted = True
                                outputGPT = "S"
                                GPTdone=True
                            else:
                                #print("Sleeping for 8.")
                                time.sleep(8)
                    currOutput = currOutput +" " + outputGPT
                #currOutput = currOutput + "\n"
                currCount+=1
                if(currCount==max):
                    writeOutput(currOutput, promptChosen,writeFilePath)
                    #print("Total Requests:", reqCount)
                    return '\n'.join([line.lstrip() for line in currOutput.splitlines()])
                else:
                    currOutput = currOutput + "\n"
            
            currLine+=1
    writeOutput(currOutput, promptChosen,writeFilePath)
    #print("Total Requests:", reqCount)
    return '\n'.join([line.lstrip() for line in currOutput.splitlines()])


In [8]:
def extractLabels(emotionFile, label, max=float('inf')):
    conversations =""
    lineNumber = 0

    with open(emotionFile, 'r') as file:
    # Loop through each line in the file
        for line in file:
            if(lineNumber == max):
                return conversations

            if(label in line):
                newLine=""
                for i in range (len(line)):
                    if((line[i] != " ") and (line[i] != "\n") and (line[i] != "1") and (line[i] != "2")):
                        newLine += '0'
                    else:
                        newLine +=line[i]
                conversations += newLine
                lineNumber+=1

        
    #print(len(conversationNumbers), count)
    return conversations

In [10]:
def writeRealLabels(labels, name):
    with open(name, 'w') as file:
        file.write(labels)

In [9]:
def processData(realLabels, genLabelsName):

    processedRealLabels=""
    processedGenLabels=""
    index=[]
    selectedIndex=[]
    indexCount=0
    n=0
    
    
    firstLine = False
    with open(genLabelsName, 'r') as file:
        for line in file:
            #print("Line at index ", indexCount,"  ", line  )
            # Skip first line
            if(firstLine==False):
                firstLine=True
            elif 'S' not in line and 'K' not in line:
                # If it doesn't contain "S", add the line to the processed content
                cleanedLine = line.replace("\n", "").replace(" ", "")
                processedGenLabels += cleanedLine
                selectedIndex.append(indexCount)
                indexCount+=1
                n+=1
                
            else:
                #indexes of lines that contain S in REAL file
                #print("Contains N!")
                index.append(indexCount)
                indexCount+=1
    
    #print(selectedIndex)
    indexCount=0
    lineCount=0
    with open(realLabels, 'r') as file:
        for line in file:
            lineCount+=1
            if ((indexCount in index) or not(indexCount in selectedIndex)):
                indexCount+=1
            else:
                indexCount+=1
                cleanedLine = line.replace("\n", "").replace(" ", "")
                for char in cleanedLine:
                    if(char =='1'):
                        processedRealLabels += char
                    else:
                        processedRealLabels += '0'            
    #print(processedRealLabels)
    #print("Original Val of n,",n)
    return processedRealLabels, processedGenLabels,n
    

In [11]:
#Interpretng Precision
#Precision is a measure of true positives / true positives + false positives
#Shows how many positives are true from the set of all elements marked as positive by the model
#TP/(TP+FP)
#5/(5+5) -> 0.5 means that out of all elements it marked as positive, only half were actually positive and the rest were negative. the higher the value, higher the precision
#higher the precision, higher the quality of positive predictions

#Interpreting Recall
#shows the proportion of all true positives that the model was able to classify as positive 
#TP/(TP + FN)
#5/(5+5) -> shows that it was only able to detect 5 out of the 10 true positive elements.
#higher the recall, higher the sensitivity
def PRFA(angryOnlyRealLabels, angryOnlyGenLabels,n, promptNumber, ITConvos, Prompt,History, Context, pathFinalOutput):
    # Initialize counters for True Positives (TP), False Positives (FP), True Negatives (TN), False Negatives (FN)
    TP = FP = TN = FN = 0
    '''
    # Calculate TP, FP, TN, FN
    for true_label, predicted_label in zip(angryOnlyRealLabels, angryOnlyGenLabels):
        if (true_label == '1' and (predicted_label == '1' or predicted_label == "2")):
            TP += 1
        elif true_label == '0' and predicted_label == '1':
            FP += 1
        elif true_label == '0' and (predicted_label == '0' or predicted_label== "2"):
            TN += 1
        elif true_label == '1' and predicted_label == '0':
            FN += 1
    '''
    for true_label, predicted_label in zip(angryOnlyRealLabels, angryOnlyGenLabels):
        if (true_label == '1' and predicted_label == '1'):
            TP += 1
        elif true_label == '0' and predicted_label == '1':
            FP += 1
        elif true_label == '0' and predicted_label == '0':
            TN += 1
        elif true_label == '1' and predicted_label == '0':
            FN += 1
    
    # Calculate Precision, Recall, F1 Score, and Accuracy
    try:
        precision = TP / (TP + FP)
        recall = TP / (TP + FN)
        f1_score = 2 * (precision * recall) / (precision + recall)
        accuracy = (TP + TN) / (TP + TN + FP + FN)
    except Exception as e:
        precision =9999999
        recall = 9999999
        f1_score =9999999
        accuracy = 9999999
        print(str(e))

    if(Context):
        Context='Y'
    else:
        Context='N'
        
    dataset = {
    "#Prompt": promptNumber,
    "#ITConvos": ITConvos,
    "Prompt": Prompt,
    "History(n)": History,
    "Context":Context,
    "Precision": precision,
    "Recall": recall,
    "F1": f1_score,
    "Accuracy": accuracy,
    "FP": FP,
    "FN": FN,
    "TP": TP,
    "TN": TN,
    "#FTMessages": FP + FN + TP + TN,
    "#FTConvos": n
    }

    df = pd.DataFrame.from_dict([dataset])
        # print("Precision:", precision)
        # print("Recall:", recall)
        # print("F1 Score:", f1_score)
        # print("Accuracy:", accuracy)
        # print("FP", FP)
        # print("FN", FN)
        # print("TP",TP)
        # print("TN",TN)
        # print("Total:", FP+FN+TP+TN)
        # print("Total Convos:",n)

    if not os.path.isfile(pathFinalOutput):
    # If the file doesn't exist, create it with the DataFrame and include the column headers
        df.to_csv(pathFinalOutput, index=False)
    else:
        # If the file exists, append the DataFrame to it without writing column headers
        df.to_csv(pathFinalOutput, mode='a', header=False, index=False)



In [12]:
#conversations with at least one message labeled as angry
angryConversations = extractEmotionConversations(emotionFile,textFile,'1')
print("Conversation indexes with at least 1 angry emotion:\n",angryConversations)

Conversation indexes with at least 1 angry emotion:
 {1, 2, 10251, 12301, 15, 32, 2084, 4132, 4134, 10281, 4143, 12337, 2100, 2101, 2102, 2106, 2111, 2113, 4161, 4165, 6214, 76, 2126, 4179, 4182, 8278, 8283, 2140, 4188, 95, 96, 97, 2148, 4196, 12395, 4205, 4208, 4211, 12403, 2168, 4216, 4217, 12408, 4220, 8318, 8321, 4227, 4228, 8324, 4231, 136, 12425, 4237, 4243, 10388, 10389, 10390, 10392, 10393, 155, 4251, 157, 10395, 10396, 4256, 4257, 4259, 4263, 6139, 2218, 10413, 12653, 2223, 2224, 4273, 178, 10415, 2228, 10416, 2230, 4278, 4280, 186, 4283, 2236, 2237, 10429, 2239, 2241, 2243, 4308, 2262, 4310, 4314, 222, 4320, 4321, 2275, 4327, 2280, 8428, 2285, 12530, 4348, 2302, 4354, 8452, 4362, 2324, 10517, 4377, 4379, 4387, 295, 296, 6440, 8487, 300, 4398, 306, 311, 8503, 4414, 10564, 2374, 327, 4423, 8521, 4426, 4427, 2380, 8522, 8526, 2383, 2384, 4431, 338, 2387, 8534, 2391, 351, 2402, 4451, 12642, 357, 364, 2413, 4460, 2415, 368, 2416, 370, 4465, 12284, 12652, 4470, 12656, 4472, 2427, 3

In [ ]:
labels = extractLabels(emotionFile, '1', 40)
writeRealLabels(labels, "angryReal40.txt")

In [665]:
labels = extractLabels(emotionFile, '1', 40)
writeRealLabels(labels, "angrydisgustReal40.txt")

In [449]:
prompts = [
    '\nState if the tone/intent of the last message in the conversation above is negative or not this format: "Y" for yes and "N" for no.',
    '\nState if the tone/intent of the last message in the conversation above is negative (even if it is slightly negative) or not. Format output like this: "Y" for yes and "N" for no.',
    '\nState if the tone/intent of the last message in the conversation above is negative (only those which are extremely negative) or not. Format output like this: "Y" for yes and "N" for no.',
    '\nState if the tone/intent of the last message in the conversation above is extremely negative, clearly not negative, or maybe negative. Format output like this: "Y" for Extremely Negative and "N" for Clearly Not Negative and "M" for Maybe Negative.',
    '\nWould the last message in the conversation above be perceived as blunt/negative/offensive by its recipient? Format your output like this: "Y" if Yes and "N" if No and "M" if Maybe.',
    '\nWould the last message in the conversation above be perceived as blunt/negative/offensive by its recipient? Format your output like this: "Y" if Yes and "N" if No.'
]

**Prompt 10**

In [511]:
prompt = 'State if the tone/intent of the last message in the conversation above is negative (even if it is slightly negative) or not. Format output like this: "Y" for yes and "N" for no.'

History: 0
Context: Yes

In [ ]:
labelsGPT = readSpecificConversations(textFile,angryConversations,prompt,"output96/output-10-1-Y.txt",history =1, context=True, max=40)

In [509]:
#reads output file
#records labels AND indexes for all convos that were NOT skipped
#reads angryReal40.txt
#extracts labels for all convos that were NOT skipped by matching their indexes with the indexes collected previously
angryOnlyRealLabels, angryOnlyGenLabels,n = processData('angryReal40.txt', 'output10/output-10-0-Y.txt')
print("Lengths:",len(angryOnlyRealLabels),len(angryOnlyGenLabels))
print("\n\n**Stats**\n\n")
PRFA(angryOnlyRealLabels,angryOnlyGenLabels,n)

Lengths: 264 264


**Stats**


Precision: 0.44871794871794873
Recall: 0.7291666666666666
F1 Score: 0.5555555555555556
Accuracy: 0.7878787878787878
FP 43
FN 13
TP 35
TN 173
Total: 264
Total Convos: 37


History: 1
Context: Y

In [512]:
labelsGPT = readSpecificConversations(textFile,angryConversations,prompt,"output10/output-10-1-Y.txt",history =1, context=True, max=40)

Skipped by GPT.

Convo:
 Person 1:  And after that ? 
Person 2:  After that , we'll let you decide if you still want a murder , I mean motorcycle . 

Skipped by GPT.

Convo:
 Person 1:  What ? He can't do this to you . 
Person 2:  Well , he did it anyway . Then he came into my room and shot all his shit blah blah blah ... you know . 

Skipped by GPT.

Skipped by GPT.

Convo:
 Person 1:  Oh my gosh . Damn ... screw the court , I would burn down their store . 
Person 2:  We'll see what happens after court . 

Total Requests: 294


In [513]:
#reads output file
#records labels AND indexes for all convos that were NOT skipped
#reads angryReal40.txt
#extracts labels for all convos that were NOT skipped by matching their indexes with the indexes collected previously
angryOnlyRealLabels, angryOnlyGenLabels,n = processData('angryReal40.txt', 'output10/output-10-1-Y.txt')
print("Lengths:",len(angryOnlyRealLabels),len(angryOnlyGenLabels))
print("\n\n**Stats**\n\n")
PRFA(angryOnlyRealLabels,angryOnlyGenLabels,n)

Lengths: 264 264


**Stats**


Precision: 0.47435897435897434
Recall: 0.7708333333333334
F1 Score: 0.5873015873015872
Accuracy: 0.803030303030303
FP 41
FN 11
TP 37
TN 175
Total: 264
Total Convos: 37


History: 2
Context: Y

In [516]:
labelsGPT = readSpecificConversations(textFile,angryConversations,prompt,"output10/output-10-2-Y.txt",history =2, context=True, max=40)

Skipped by GPT.

Convo:
 Person 1: Mom , I want to get a motorcycle . Is that all right with you ? '
Person 2:  You mean a murder cycle ? Did you know more cyclists die in accidents than motorists ? '
Person 1:  Mom ! I'll wear a helmet and I'll drive sane . I promise . '
Person 2:  I'll tell you what . You can get a motorcycle on one condition . '
Person 1:  What's that ? '
Person 2:  You spend two weeks with dad in his ambulance as an EMT trainee and volunteer at the emergency room for one month . '
Person 1:  And after that ? '
Person 2:  After that , we'll let you decide if you still want a murder , I mean motorcycle . '

Skipped by GPT.

Convo:
 Person 1: What's up ? You sound a little down in dumps . '
Person 2:  I quarreled with my roommate last night . He is really stubborn . '
Person 1:  Calm down ! Shouting won ’ t help ? '
Person 2:  He is really outrageous . '
Person 1:  What happened ? '
Person 2:  I went back home last night . You know tired as a dog , so I took a quick s

In [518]:
#reads output file
#records labels AND indexes for all convos that were NOT skipped
#reads angryReal40.txt
#extracts labels for all convos that were NOT skipped by matching their indexes with the indexes collected previously
angryOnlyRealLabels, angryOnlyGenLabels,n = processData('angryReal40.txt', 'output10/output-10-2-Y.txt')
print("Lengths:",len(angryOnlyRealLabels),len(angryOnlyGenLabels))
print("\n\n**Stats**\n\n")
PRFA(angryOnlyRealLabels,angryOnlyGenLabels,n)

Lengths: 274 274


**Stats**


Precision: 0.42857142857142855
Recall: 0.78
F1 Score: 0.5531914893617021
Accuracy: 0.7700729927007299
FP 52
FN 11
TP 39
TN 172
Total: 274
Total Convos: 38


History: 0
Context: N

In [522]:
labelsGPT = readSpecificConversations(textFile,angryConversations,prompt,"output10/output-10-0-N.txt",history =0, context=False, max=40)

Skipped by GPT.

Convo:
 Person 2:  After that , we'll let you decide if you still want a murder , I mean motorcycle . 
Skipped by GPT.

Convo:
 Person 2:  Well , he did it anyway . Then he came into my room and shot all his shit blah blah blah ... you know . 
Skipped by GPT.

Convo:
 Person 1:  Oh my gosh . Damn ... screw the court , I would burn down their store . 
Total Requests: 295


In [523]:
#reads output file
#records labels AND indexes for all convos that were NOT skipped
#reads angryReal40.txt
#extracts labels for all convos that were NOT skipped by matching their indexes with the indexes collected previously
angryOnlyRealLabels, angryOnlyGenLabels,n = processData('angryReal40.txt', 'output10/output-10-0-N.txt')
print("Lengths:",len(angryOnlyRealLabels),len(angryOnlyGenLabels))
print("\n\n**Stats**\n\n")
PRFA(angryOnlyRealLabels,angryOnlyGenLabels,n)

Lengths: 264 264


**Stats**


Precision: 0.4375
Recall: 0.7291666666666666
F1 Score: 0.546875
Accuracy: 0.7803030303030303
FP 45
FN 13
TP 35
TN 171
Total: 264
Total Convos: 37


History: 1
Context: N

In [524]:
labelsGPT = readSpecificConversations(textFile,angryConversations,prompt,"output10/output-10-1-N.txt",history =1, context=False, max=40)

Skipped by GPT.

Convo:
 Person 2:  What ? 
Person 1:  Any body piercing ? Stick out your tongue ! 

Skipped by GPT.

Convo:
 Person 1:  And after that ? 
Person 2:  After that , we'll let you decide if you still want a murder , I mean motorcycle . 

Skipped by GPT.

Convo:
 Person 1:  What ? He can't do this to you . 
Person 2:  Well , he did it anyway . Then he came into my room and shot all his shit blah blah blah ... you know . 

Skipped by GPT.

Skipped by GPT.

Convo:
 Person 1:  Oh my gosh . Damn ... screw the court , I would burn down their store . 
Person 2:  We'll see what happens after court . 

Total Requests: 293


In [525]:
#reads output file
#records labels AND indexes for all convos that were NOT skipped
#reads angryReal40.txt
#extracts labels for all convos that were NOT skipped by matching their indexes with the indexes collected previously
angryOnlyRealLabels, angryOnlyGenLabels,n = processData('angryReal40.txt', 'output10/output-10-1-N.txt')
print("Lengths:",len(angryOnlyRealLabels),len(angryOnlyGenLabels))
print("\n\n**Stats**\n\n")
PRFA(angryOnlyRealLabels,angryOnlyGenLabels,n)

Lengths: 256 256


**Stats**


Precision: 0.46153846153846156
Recall: 0.7659574468085106
F1 Score: 0.5760000000000001
Accuracy: 0.79296875
FP 42
FN 11
TP 36
TN 167
Total: 256
Total Convos: 36


History: 2 Context: N

In [526]:
labelsGPT = readSpecificConversations(textFile,angryConversations,prompt,"output10/output-10-2-N.txt",history =2, context=False, max=40)

Skipped by GPT.

Convo:
 Person 1: Mom , I want to get a motorcycle . Is that all right with you ? '
Person 2:  You mean a murder cycle ? Did you know more cyclists die in accidents than motorists ? '
Person 1:  Mom ! I'll wear a helmet and I'll drive sane . I promise . '
Person 2:  I'll tell you what . You can get a motorcycle on one condition . '
Person 1:  What's that ? '
Person 2:  You spend two weeks with dad in his ambulance as an EMT trainee and volunteer at the emergency room for one month . '
Person 1:  And after that ? '
Person 2:  After that , we'll let you decide if you still want a murder , I mean motorcycle . '

Skipped by GPT.

Convo:
 Person 1: What's up ? You sound a little down in dumps . '
Person 2:  I quarreled with my roommate last night . He is really stubborn . '
Person 1:  Calm down ! Shouting won ’ t help ? '
Person 2:  He is really outrageous . '
Person 1:  What happened ? '
Person 2:  I went back home last night . You know tired as a dog , so I took a quick s

In [527]:
#reads output file
#records labels AND indexes for all convos that were NOT skipped
#reads angryReal40.txt
#extracts labels for all convos that were NOT skipped by matching their indexes with the indexes collected previously
angryOnlyRealLabels, angryOnlyGenLabels,n = processData('angryReal40.txt', 'output10/output-10-2-N.txt')
print("Lengths:",len(angryOnlyRealLabels),len(angryOnlyGenLabels))
print("\n\n**Stats**\n\n")
PRFA(angryOnlyRealLabels,angryOnlyGenLabels,n)

Lengths: 259 259


**Stats**


Precision: 0.4523809523809524
Recall: 0.7755102040816326
F1 Score: 0.5714285714285715
Accuracy: 0.7799227799227799
FP 46
FN 11
TP 38
TN 164
Total: 259
Total Convos: 37


Prompt 11

In [528]:
prompt = 'State if the tone/intent of the last message in the conversation above is negative (only if it is extremely negative) or not. Format output like this: "Y" for yes and "N" for no.'

History:0 Context: Yes

In [530]:
labelsGPT = readSpecificConversations(textFile,angryConversations,prompt,"output11/output-11-0-Y.txt",history =0, context=True, max=40)

Skipped by GPT.

Convo:
 Person 2:  After that , we'll let you decide if you still want a murder , I mean motorcycle . 
Skipped by GPT.

Convo:
 Person 2:  Well , he did it anyway . Then he came into my room and shot all his shit blah blah blah ... you know . 
K Produced As an AI, I need the conversation above to analyze and answer your request, which you didn't provide. Could you please provide the conversation to identify whether the final message in it has a negative tone/intent or not? 
Prompt: The topic of the following conversation is Ordinary Life.
Person 1:  What about this ? State if the tone/intent of the last message in the conversation above is negative (only if it is extremely negative) or not. Format output like this: "Y" for yes and "N" for no.
Skipped by GPT.

Convo:
 Person 1:  Oh my gosh . Damn ... screw the court , I would burn down their store . 
Total Requests: 295


In [531]:
angryOnlyRealLabels, angryOnlyGenLabels,n = processData('angryReal40.txt', 'output11/output-11-0-Y.txt')
print("Lengths:",len(angryOnlyRealLabels),len(angryOnlyGenLabels))
print("\n\n**Stats**\n\n")
PRFA(angryOnlyRealLabels,angryOnlyGenLabels,n)

Lengths: 264 264


**Stats**


Precision: 0.6666666666666666
Recall: 0.4583333333333333
F1 Score: 0.5432098765432098
Accuracy: 0.8593155893536122
FP 11
FN 26
TP 22
TN 204
Total: 263
Total Convos: 37


History: 1 Context: Yes

In [532]:
labelsGPT = readSpecificConversations(textFile,angryConversations,prompt,"output11/output-11-1-Y.txt",history =1, context=True, max=40)

Skipped by GPT.

Convo:
 Person 1:  And after that ? 
Person 2:  After that , we'll let you decide if you still want a murder , I mean motorcycle . 

Skipped by GPT.

Convo:
 Person 1:  What ? He can't do this to you . 
Person 2:  Well , he did it anyway . Then he came into my room and shot all his shit blah blah blah ... you know . 

Skipped by GPT.

Skipped by GPT.

Convo:
 Person 1:  Oh my gosh . Damn ... screw the court , I would burn down their store . 
Person 2:  We'll see what happens after court . 

Total Requests: 294


In [534]:
angryOnlyRealLabels, angryOnlyGenLabels,n = processData('angryReal40.txt', 'output11/output-11-1-Y.txt')
print("Lengths:",len(angryOnlyRealLabels),len(angryOnlyGenLabels))
print("\n\n**Stats**\n\n")
PRFA(angryOnlyRealLabels,angryOnlyGenLabels,n)

Lengths: 264 264


**Stats**


Precision: 0.6071428571428571
Recall: 0.3541666666666667
F1 Score: 0.4473684210526316
Accuracy: 0.8409090909090909
FP 11
FN 31
TP 17
TN 205
Total: 264
Total Convos: 37


History: 2 Context Yes

In [535]:
labelsGPT = readSpecificConversations(textFile,angryConversations,prompt,"output11/output-11-2-Y.txt",history =2, context=True, max=40)

Skipped by GPT.

Convo:
 Person 1: Mom , I want to get a motorcycle . Is that all right with you ? '
Person 2:  You mean a murder cycle ? Did you know more cyclists die in accidents than motorists ? '
Person 1:  Mom ! I'll wear a helmet and I'll drive sane . I promise . '
Person 2:  I'll tell you what . You can get a motorcycle on one condition . '
Person 1:  What's that ? '
Person 2:  You spend two weeks with dad in his ambulance as an EMT trainee and volunteer at the emergency room for one month . '
Person 1:  And after that ? '
Person 2:  After that , we'll let you decide if you still want a murder , I mean motorcycle . '

Skipped by GPT.

Convo:
 Person 1: What's up ? You sound a little down in dumps . '
Person 2:  I quarreled with my roommate last night . He is really stubborn . '
Person 1:  Calm down ! Shouting won ’ t help ? '
Person 2:  He is really outrageous . '
Person 1:  What happened ? '
Person 2:  I went back home last night . You know tired as a dog , so I took a quick s

In [536]:
angryOnlyRealLabels, angryOnlyGenLabels,n = processData('angryReal40.txt', 'output11/output-11-2-Y.txt')
print("Lengths:",len(angryOnlyRealLabels),len(angryOnlyGenLabels))
print("\n\n**Stats**\n\n")
PRFA(angryOnlyRealLabels,angryOnlyGenLabels,n)

Lengths: 274 274


**Stats**


Precision: 0.6571428571428571
Recall: 0.46
F1 Score: 0.5411764705882354
Accuracy: 0.8576642335766423
FP 12
FN 27
TP 23
TN 212
Total: 274
Total Convos: 38


History: 0 Context No

In [537]:
labelsGPT = readSpecificConversations(textFile,angryConversations,prompt,"output11/output-11-0-N.txt",history =0, context=False, max=40)

Skipped by GPT.

Convo:
 Person 2:  After that , we'll let you decide if you still want a murder , I mean motorcycle . 
Skipped by GPT.

Convo:
 Person 2:  Well , he did it anyway . Then he came into my room and shot all his shit blah blah blah ... you know . 
Skipped by GPT.

Convo:
 Person 1:  Oh my gosh . Damn ... screw the court , I would burn down their store . 
Total Requests: 295


In [539]:
angryOnlyRealLabels, angryOnlyGenLabels,n = processData('angryReal40.txt', 'output11/output-11-0-N.txt')
print("Lengths:",len(angryOnlyRealLabels),len(angryOnlyGenLabels))
print("\n\n**Stats**\n\n")
PRFA(angryOnlyRealLabels,angryOnlyGenLabels,n)

Lengths: 264 264


**Stats**


Precision: 0.6666666666666666
Recall: 0.4166666666666667
F1 Score: 0.5128205128205129
Accuracy: 0.8560606060606061
FP 10
FN 28
TP 20
TN 206
Total: 264
Total Convos: 37


History: 1 Context No

In [540]:
labelsGPT = readSpecificConversations(textFile,angryConversations,prompt,"output11/output-11-1-N.txt",history =1, context=False, max=40)

Skipped by GPT.

Convo:
 Person 2:  What ? 
Person 1:  Any body piercing ? Stick out your tongue ! 

Skipped by GPT.

Convo:
 Person 1:  And after that ? 
Person 2:  After that , we'll let you decide if you still want a murder , I mean motorcycle . 

Skipped by GPT.

Convo:
 Person 1:  What ? He can't do this to you . 
Person 2:  Well , he did it anyway . Then he came into my room and shot all his shit blah blah blah ... you know . 

Skipped by GPT.

Skipped by GPT.

Convo:
 Person 1:  Oh my gosh . Damn ... screw the court , I would burn down their store . 
Person 2:  We'll see what happens after court . 

Total Requests: 293


In [541]:
angryOnlyRealLabels, angryOnlyGenLabels,n = processData('angryReal40.txt', 'output11/output-11-1-N.txt')
print("Lengths:",len(angryOnlyRealLabels),len(angryOnlyGenLabels))
print("\n\n**Stats**\n\n")
PRFA(angryOnlyRealLabels,angryOnlyGenLabels,n)

Lengths: 256 256


**Stats**


Precision: 0.6896551724137931
Recall: 0.425531914893617
F1 Score: 0.5263157894736842
Accuracy: 0.859375
FP 9
FN 27
TP 20
TN 200
Total: 256
Total Convos: 36


History: 2 Context No

In [542]:
labelsGPT = readSpecificConversations(textFile,angryConversations,prompt,"output11/output-11-2-N.txt",history =2, context=False, max=40)

Skipped by GPT.

Convo:
 Person 1: Mom , I want to get a motorcycle . Is that all right with you ? '
Person 2:  You mean a murder cycle ? Did you know more cyclists die in accidents than motorists ? '
Person 1:  Mom ! I'll wear a helmet and I'll drive sane . I promise . '
Person 2:  I'll tell you what . You can get a motorcycle on one condition . '
Person 1:  What's that ? '
Person 2:  You spend two weeks with dad in his ambulance as an EMT trainee and volunteer at the emergency room for one month . '
Person 1:  And after that ? '
Person 2:  After that , we'll let you decide if you still want a murder , I mean motorcycle . '

Skipped by GPT.

Convo:
 Person 1: What's up ? You sound a little down in dumps . '
Person 2:  I quarreled with my roommate last night . He is really stubborn . '
Person 1:  Calm down ! Shouting won ’ t help ? '
Person 2:  He is really outrageous . '
Person 1:  What happened ? '
Person 2:  I went back home last night . You know tired as a dog , so I took a quick s

In [543]:
angryOnlyRealLabels, angryOnlyGenLabels,n = processData('angryReal40.txt', 'output11/output-11-2-N.txt')
print("Lengths:",len(angryOnlyRealLabels),len(angryOnlyGenLabels))
print("\n\n**Stats**\n\n")
PRFA(angryOnlyRealLabels,angryOnlyGenLabels,n)

Lengths: 259 259


**Stats**


Precision: 0.6875
Recall: 0.4489795918367347
F1 Score: 0.5432098765432098
Accuracy: 0.8571428571428571
FP 10
FN 27
TP 22
TN 200
Total: 259
Total Convos: 37


Automated 

In [48]:
historyOptions=[2]
contextOptions=[False]
promptNo=["55.64","55.65","55.66"]
promptOptions=['State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \n @rukhshan@ \nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: "Y" if yes and "N" if no.',
               'State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \n @rukhshan@ \nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: "Y" if yes and "N" if no.',
               'State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \n @rukhshan@ \nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: "Y" if yes and "N" if no.']

groundPath = "/Users/rukhshanharoon/Desktop/LLMChattingApp/Analysis/angryReal40.txt"
maxCon=40

pathFinalOutput="/Users/rukhshanharoon/Desktop/LLMChattingApp/Analysis/finalOutput.csv"

if(len(promptNo)!=len(promptOptions)):
    raise ValueError("promptNo not equal to promptOptions!")


for promptIndex in range(len(promptOptions)):
    for historySelected in historyOptions:
        for contextSelected in contextOptions:
            folderName = "/Users/rukhshanharoon/Desktop/LLMChattingApp/Analysis/output"+promptNo[promptIndex]
            if not os.path.exists(folderName):
                os.makedirs(folderName)
            fileName = "output-"+promptNo[promptIndex]+"-"+str(historySelected)+"-"+ ('Y.txt' if contextSelected else 'N.txt')
            genPath = folderName + "/" + fileName
            labelsGPT = readSpecificConversations(textFile,angryConversations,promptOptions[promptIndex],genPath,historySelected, contextSelected, maxCon)
            angryOnlyRealLabels, angryOnlyGenLabels,n = processData(groundPath, genPath)
            print("N is", n)
            if(len(angryOnlyRealLabels) != len(angryOnlyGenLabels)):
                print("Length of ground and gen not equal!")
                print(len(angryOnlyRealLabels), len(angryOnlyGenLabels))
                print("Issue in: ", fileName)
            else:
                df = PRFA(angryOnlyRealLabels,angryOnlyGenLabels,n, promptNo[promptIndex],maxCon, promptOptions[promptIndex],historySelected,contextSelected, pathFinalOutput)
                print(fileName, "done!")

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Exception: Requests to the Creates a completion for the chat message Operation under Azure OpenAI API version 2023-05-15 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 5 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Exception: Requests to the Creates a completion for the chat message Operation under Azure OpenAI API version 2023-05-15 have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 5 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Exception: Requests to the Creates a completion for the chat message Operation under Azure OpenAI API version 2023-05-15 have exceeded call rate limit of your current OpenAI S0 pricing t

In [15]:
def printDialogue(index, filePath="/Users/rukhshanharoon/Desktop/LLMChattingApp/Analysis/Dataset/ijcnlp_dailydialog/dialogues_text.txt"):
    personNumber=1
    with open(filePath, 'r') as file:
        lines = file.readlines()
        if 0 <= index <= len(lines):
            convo = lines[index].split("__eou__")
            for utterance in convo:
                print("Person#"+str(personNumber)+" " +utterance)
                if(personNumber==1):
                    personNumber=2
                else:
                    personNumber=1
        else:
            print("Line number is out of range.")

#conversations with at least one message labeled as angry
angryConversations = extractEmotionConversations(emotionFile,textFile,'1')
angryConversations = sorted(list(angryConversations))
print(angryConversations[0:45])
len(angryConversations)



[1, 2, 15, 32, 76, 95, 96, 97, 136, 155, 157, 178, 186, 222, 295, 296, 300, 306, 311, 327, 338, 351, 357, 364, 368, 370, 380, 392, 408, 417, 490, 519, 529, 576, 636, 769, 789, 814, 917, 935, 967, 991, 1042, 1176, 1194]


508

In [12]:
def checkDiff(realFile, genFile, flag = 'Y'):
    linesGenList = []
    indexGenList=[]
    index=-1
    linesRealList=[]
    with open(genFile, 'r') as file:
    # Loop through each line in the file
        firstLine=False
        for line in file:
            index+=1
            if(firstLine == False):
                firstLine=True
                index-=1
            elif("S" in line or "K" in line):
                continue
            else:
                linesGenList.append(line.strip().replace(" ",""))
                indexGenList.append(index)
                
    with open(realFile) as file:
        linesRealList = list(file)
        for i in range(len(linesRealList)):
            linesRealList[i]=linesRealList[i].strip()
            linesRealList[i]=linesRealList[i].replace(" ","")
   
    #print(indexGenList)
    
    for i in range(len(indexGenList)):
        for messageI in range(len(linesGenList[i])):
            if(flag =='Y'):
                if(linesRealList[indexGenList[i]][messageI]=='1' and linesGenList[i][messageI]=='0'):
                    print("Real",linesRealList[indexGenList[i]][messageI],"Gen",linesGenList[i][messageI])
                    print("Convo at index# (line gen): ",indexGenList[i]+2,"Gen:",linesGenList[i], "Real:", linesRealList[indexGenList[i]],"Message I:", messageI)
            else:
                if(linesRealList[indexGenList[i]][messageI]=='0' and linesGenList[i][messageI]=='1'):
                    print("Real",linesRealList[indexGenList[i]][messageI],"Gen",linesGenList[i][messageI])
                    print("Convo at index# (line gen): ",indexGenList[i]+2,"Gen:",linesGenList[i], "Real:", linesRealList[indexGenList[i]],"Message I:", messageI)


                
#printing line# in gen file
#l# in real file = line gen - 1
#index for printing = gen file - 2

checkDiff("angryReal40.txt","output40/output-40-0-N.txt")
print("***")
checkDiff("angryReal40.txt","output40/output-40-1-N.txt")
print("***")
checkDiff("angryReal40.txt","output40/output-40-2-N.txt")
#checkDiff("angryReal40.txt","output54.7/output-54.7-2-N.txt")
    

Real 1 Gen 0
Convo at index# (line gen):  2 Gen: 01101 Real: 00010 Message I: 3
Real 1 Gen 0
Convo at index# (line gen):  4 Gen: 010100 Real: 010101 Message I: 5
Real 1 Gen 0
Convo at index# (line gen):  15 Gen: 00000 Real: 00001 Message I: 4
Real 1 Gen 0
Convo at index# (line gen):  18 Gen: 01011010 Real: 00000100 Message I: 5
Real 1 Gen 0
Convo at index# (line gen):  20 Gen: 00111101 Real: 00000010 Message I: 6
Real 1 Gen 0
Convo at index# (line gen):  21 Gen: 0101010101 Real: 0000001000 Message I: 6
Real 1 Gen 0
Convo at index# (line gen):  23 Gen: 10101001 Real: 00000101 Message I: 5
Real 1 Gen 0
Convo at index# (line gen):  26 Gen: 101001101 Real: 000101000 Message I: 3
Real 1 Gen 0
Convo at index# (line gen):  31 Gen: 0000000100 Real: 0000000001 Message I: 9
Real 1 Gen 0
Convo at index# (line gen):  38 Gen: 000 Real: 001 Message I: 2
***
Real 1 Gen 0
Convo at index# (line gen):  2 Gen: 01100 Real: 00010 Message I: 3
Real 1 Gen 0
Convo at index# (line gen):  4 Gen: 010100 Real: 01

Checking False Positives (flag = 'N')

In [13]:
checkDiff("angryReal40.txt","output55.36/output-55.36-2-N.txt",'N')
print("***")

Real 0 Gen 1
Convo at index# (line gen):  2 Gen: 01111 Real: 00010 Message I: 1
Real 0 Gen 1
Convo at index# (line gen):  2 Gen: 01111 Real: 00010 Message I: 2
Real 0 Gen 1
Convo at index# (line gen):  2 Gen: 01111 Real: 00010 Message I: 4
Real 0 Gen 1
Convo at index# (line gen):  3 Gen: 1110 Real: 0100 Message I: 0
Real 0 Gen 1
Convo at index# (line gen):  3 Gen: 1110 Real: 0100 Message I: 2
Real 0 Gen 1
Convo at index# (line gen):  4 Gen: 011101 Real: 010101 Message I: 2
Real 0 Gen 1
Convo at index# (line gen):  5 Gen: 11 Real: 01 Message I: 0
Real 0 Gen 1
Convo at index# (line gen):  6 Gen: 11 Real: 01 Message I: 0
Real 0 Gen 1
Convo at index# (line gen):  7 Gen: 1111010 Real: 0100000 Message I: 0
Real 0 Gen 1
Convo at index# (line gen):  7 Gen: 1111010 Real: 0100000 Message I: 2
Real 0 Gen 1
Convo at index# (line gen):  7 Gen: 1111010 Real: 0100000 Message I: 3
Real 0 Gen 1
Convo at index# (line gen):  7 Gen: 1111010 Real: 0100000 Message I: 5
Real 0 Gen 1
Convo at index# (line gen

In [16]:
#124
printDialogue(angryConversations[0])
print("**********")

Person#1 So Dick , how about getting some coffee for tonight ? 
Person#2  Coffee ? I don ’ t honestly like that kind of stuff . 
Person#1  Come on , you can at least try a little , besides your cigarette . 
Person#2  What ’ s wrong with that ? Cigarette is the thing I go crazy for . 
Person#1  Not for me , Dick . 
Person#2 

**********


In [17]:
#1
printDialogue(angryConversations[8])
print("**********")


Person#1 I stayed at Bob's last night . 
Person#2  I was worried where you were . 
Person#1  I guess I should have told you . 
Person#2  Yes , you should have . 
Person#1 

**********


In [42]:
message ="I was worried where you were ."

conv1 = "Here is a conversation: \nPerson#1 I stayed at Bob's last night . \nPerson#2  I was worried where you were ."
prompt1 = conv1 + '\nState if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient:\n' + message + '\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: "Y" if yes and "N" if no.'

messages = [
    {"role":"user","content":"Here is a conversation: \nPerson#1 Ouch ! I'm hurt . \nPerson#2  Are you all right ?\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nAre you all right ?\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},

    {"role":"user","content":"Here is a conversation: \nPerson#1 Ouch ! I'm hurt . \nPerson#2  Are you all right ? \nPerson#1  Yes . I'm OK . I just had a tumble . No big deal. \nPerson#2  Good . You scared me . \n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nGood . You scared me . \n Take into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},

    {"role":"user","content":"Here is a conversation: \nPerson#1 Ouch ! I'm hurt . \nPerson#2  Are you all right ? \nPerson#1  Yes . I'm OK . I just had a tumble . No big deal. \nPerson#2  Good . You scared me . \nPerson#1  Sorry . Can you please help me up ! I have trouble standing up by myself with the skis on . \n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nSorry . Can you please help me up ! I have trouble standing up by myself with the skis on .\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},



    {"role":"user","content":"Here is a conversation: \nPerson#1 What's the fare , please ?\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nWhat's the fare , please? \nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},

    {"role":"user","content":"Here is a conversation: \nPerson#1 What's the fare , please ?\nPerson#2  Twenty dollars and sixty cents .\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nTwenty dollars and sixty cents .\n Take into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},

    {"role":"user","content":"Here is a conversation: \nPerson#1 What's the fare , please ?\nPerson#2  Twenty dollars and sixty cents .\nPerson#1  What ? You're overcharging me !\nPerson#2  No way . Look . That's what the meter says .\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nNo way . Look . That's what the meter says .\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},
    {"role": "user", "content": prompt1},
    {"role": "assistant","content": "Y"},
    {"role": "user", "content": "Justify why you output 'Y', indicating it is negative. Humans labelled it as not negative."}
 ]
callGPTTesting(messages)


Apologies for any confusion, the sentence "I was worried where you were" expresses concern for the other person's well-being, which is generally seen as positive. However, it can be slightly negative only due to the fact that it implies the speaker was anxious or stressed because they didn't know where the other person was. But that's open to interpretation, based on context and personal perception. In the context provided, it is mostly seen as a positive statement of caring therefore the correct answer should be 'N'.


In [19]:

printDialogue(angryConversations[18])
print("**********")
#01457

Person#1 Mom , are those tattoos on your brows ? 
Person#2  Uh , yes , they are . 
Person#1  Moooooommmmm ! You're so in ! Anything else you've been keeping from me ? 
Person#2  What ? 
Person#1  Any body piercing ? Stick out your tongue ! 
Person#2  Oh , for Pete's sake . This is cosmetic . My brows are falling out . I did this for convenience . 
Person#1  Come on mom . I'm your A ! Wait till the guys hear about this ! 
Person#2  Mike , don't even think about it ! 
Person#1 

**********


In [41]:
#0
message="Person#1 Mom , are those tattoos on your brows ?\n"

conv1 = "Here is a conversation: \nPerson#1 Mom , are those tattoos on your brows ?\n"
prompt1 = conv1 + '\nState if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient:\n' + message + '\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: "Y" if yes and "N" if no.'

messages = [
    {"role":"user","content":"Here is a conversation: \nPerson#1 Ouch ! I'm hurt . \nPerson#2  Are you all right ?\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nAre you all right ?\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},

    {"role":"user","content":"Here is a conversation: \nPerson#1 Ouch ! I'm hurt . \nPerson#2  Are you all right ? \nPerson#1  Yes . I'm OK . I just had a tumble . No big deal. \nPerson#2  Good . You scared me . \n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nGood . You scared me . \n Take into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},

    {"role":"user","content":"Here is a conversation: \nPerson#1 Ouch ! I'm hurt . \nPerson#2  Are you all right ? \nPerson#1  Yes . I'm OK . I just had a tumble . No big deal. \nPerson#2  Good . You scared me . \nPerson#1  Sorry . Can you please help me up ! I have trouble standing up by myself with the skis on . \n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nSorry . Can you please help me up ! I have trouble standing up by myself with the skis on .\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},



    {"role":"user","content":"Here is a conversation: \nPerson#1 What's the fare , please ?\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nWhat's the fare , please? \nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},

    {"role":"user","content":"Here is a conversation: \nPerson#1 What's the fare , please ?\nPerson#2  Twenty dollars and sixty cents .\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nTwenty dollars and sixty cents .\n Take into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},

    {"role":"user","content":"Here is a conversation: \nPerson#1 What's the fare , please ?\nPerson#2  Twenty dollars and sixty cents .\nPerson#1  What ? You're overcharging me !\nPerson#2  No way . Look . That's what the meter says .\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nNo way . Look . That's what the meter says .\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},
    {"role": "user", "content": prompt1},
    {"role": "assistant","content": "Y"},
    {"role": "user", "content": "Justify why you output 'Y', indicating it is negative. Humans labelled it as not negative."}
 ]
callGPTTesting(messages)
#I do not completely agree with GPT's explanation as there is no indication of surprise (could be communicated through !! or relevant emojis)
#Note that GPT's explanation may vary if rerun.

Given the lack of context and specific tone, the comment could be seen as neutral. However, the question "Mom, are those tattoos on your brows?" could potentially be taken negatively depending on the mother's perception. It may imply surprise or disapproval of the mother's appearance choices. The degree of negativity would ultimately depend on the personal interpretations and sensitivity of the specific individuals involved in the conversation.


In [43]:
#5

message=" Oh , for Pete's sake . This is cosmetic . My brows are falling out . I did this for convenience .\n"

conv1 = "Here is a conversation: \nPerson#1 Mom , are those tattoos on your brows ? \nPerson#2  Uh , yes , they are . \nPerson#1  Moooooommmmm ! You're so in ! Anything else you've been keeping from me ? \nPerson#2  What ? \nPerson#1  Any body piercing ? Stick out your tongue ! \nPerson#2  Oh , for Pete's sake . This is cosmetic . My brows are falling out . I did this for convenience . \n"
prompt1 = conv1 + '\nState if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient:\n' + message + '\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: "Y" if yes and "N" if no.'

messages = [
    {"role":"user","content":"Here is a conversation: \nPerson#1 Ouch ! I'm hurt . \nPerson#2  Are you all right ?\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nAre you all right ?\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},

    {"role":"user","content":"Here is a conversation: \nPerson#1 Ouch ! I'm hurt . \nPerson#2  Are you all right ? \nPerson#1  Yes . I'm OK . I just had a tumble . No big deal. \nPerson#2  Good . You scared me . \n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nGood . You scared me . \n Take into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},

    {"role":"user","content":"Here is a conversation: \nPerson#1 Ouch ! I'm hurt . \nPerson#2  Are you all right ? \nPerson#1  Yes . I'm OK . I just had a tumble . No big deal. \nPerson#2  Good . You scared me . \nPerson#1  Sorry . Can you please help me up ! I have trouble standing up by myself with the skis on . \n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nSorry . Can you please help me up ! I have trouble standing up by myself with the skis on .\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},



    {"role":"user","content":"Here is a conversation: \nPerson#1 What's the fare , please ?\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nWhat's the fare , please? \nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},

    {"role":"user","content":"Here is a conversation: \nPerson#1 What's the fare , please ?\nPerson#2  Twenty dollars and sixty cents .\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nTwenty dollars and sixty cents .\n Take into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},

    {"role":"user","content":"Here is a conversation: \nPerson#1 What's the fare , please ?\nPerson#2  Twenty dollars and sixty cents .\nPerson#1  What ? You're overcharging me !\nPerson#2  No way . Look . That's what the meter says .\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nNo way . Look . That's what the meter says .\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},
    {"role": "user", "content": prompt1},
    {"role": "assistant","content": "Y"},
    {"role": "user", "content": "Justify why you output 'Y', indicating it is negative. Humans labelled it as not negative."}
 ]
callGPTTesting(messages)
#I do not completely agree with GPT's explanation as there is no indication of surprise (could be communicated through !! or relevant emojis)
#Note that GPT's explanation may vary if rerun.

The phrase "Oh, for Pete's sake" can potentially be seen as a minor sign of frustration or annoyance with the preceding question, which may be perceived as slightly negative. Although it seems to be a mild rebuke rather than a serious reprimand, different recipients could interpret this differently.


In [32]:
print(angryConversations)
printDialogue(angryConversations[28])

print("**********")
#01




[1, 2, 15, 32, 76, 95, 96, 97, 136, 155, 157, 178, 186, 222, 295, 296, 300, 306, 311, 327, 338, 351, 357, 364, 368, 370, 380, 392, 408, 417, 490, 519, 529, 576, 636, 769, 789, 814, 917, 935, 967, 991, 1042, 1176, 1194, 1245, 1301, 1319, 1347, 1348, 1364, 1377, 1423, 1428, 1429, 1443, 1483, 1494, 1504, 1525, 1530, 1535, 1539, 1542, 1543, 1595, 1598, 1654, 1668, 1741, 1752, 1779, 1791, 1799, 1828, 1922, 1940, 1969, 1977, 1999, 2026, 2029, 2033, 2043, 2084, 2100, 2101, 2102, 2106, 2111, 2113, 2126, 2140, 2148, 2168, 2218, 2223, 2224, 2228, 2230, 2236, 2237, 2239, 2241, 2243, 2262, 2275, 2280, 2285, 2302, 2324, 2374, 2380, 2383, 2384, 2387, 2391, 2402, 2413, 2415, 2416, 2427, 2435, 2451, 2471, 2474, 2476, 2478, 2489, 2505, 2509, 2510, 2515, 2518, 2523, 2524, 2525, 2526, 2534, 2617, 2651, 2654, 2657, 2661, 2687, 2728, 2758, 2785, 2811, 2816, 2887, 2902, 2924, 2937, 2992, 3006, 3017, 3023, 3030, 3042, 3050, 3073, 3077, 3078, 3084, 3085, 3087, 3091, 3092, 3095, 3100, 3102, 3105, 3106, 3109, 3

In [40]:
#0
message="Tom , it's time for you to take a shower ."

conv1 = "Here is a conversation: \nPerson#1 Tom , it's time for you to take a shower .\n"
prompt1 = conv1 + '\nState if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient:\n' + message + '\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: "Y" if yes and "N" if no.'

messages = [
    {"role":"user","content":"Here is a conversation: \nPerson#1 Ouch ! I'm hurt . \nPerson#2  Are you all right ?\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nAre you all right ?\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},

    {"role":"user","content":"Here is a conversation: \nPerson#1 Ouch ! I'm hurt . \nPerson#2  Are you all right ? \nPerson#1  Yes . I'm OK . I just had a tumble . No big deal. \nPerson#2  Good . You scared me . \n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nGood . You scared me . \n Take into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},

    {"role":"user","content":"Here is a conversation: \nPerson#1 Ouch ! I'm hurt . \nPerson#2  Are you all right ? \nPerson#1  Yes . I'm OK . I just had a tumble . No big deal. \nPerson#2  Good . You scared me . \nPerson#1  Sorry . Can you please help me up ! I have trouble standing up by myself with the skis on . \n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nSorry . Can you please help me up ! I have trouble standing up by myself with the skis on .\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},



    {"role":"user","content":"Here is a conversation: \nPerson#1 What's the fare , please ?\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nWhat's the fare , please? \nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},

    {"role":"user","content":"Here is a conversation: \nPerson#1 What's the fare , please ?\nPerson#2  Twenty dollars and sixty cents .\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nTwenty dollars and sixty cents .\n Take into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},

    {"role":"user","content":"Here is a conversation: \nPerson#1 What's the fare , please ?\nPerson#2  Twenty dollars and sixty cents .\nPerson#1  What ? You're overcharging me !\nPerson#2  No way . Look . That's what the meter says .\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nNo way . Look . That's what the meter says .\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},
    {"role": "user", "content": prompt1},
    {"role": "assistant","content": "Y"},
    {"role": "user", "content": "Justify why you output 'Y', indicating it is negative. Humans labelled it as not negative."}
 ]
callGPTTesting(messages)


The conversation message "Tom , it's time for you to take a shower ." suggests that Person#1 is asking or directing Person#2 (Tom) to do something, which might be perceived as an order, command or at least, an imposition depending on their relationship. It could be seen as negative if Tom doesn't want to take a shower at this time or if he feels he is being nagged. The negativity here is subtle and depends largely on the tone of the speaker and their relationship with Tom.


In [39]:
#1
message="Mom , I don't want to bathe today . "

conv1 = "Here is a conversation: \nPerson#1 Tom , it's time for you to take a shower .\nPerson#2  Mom , I don't want to bathe today . \n"
prompt1 = conv1 + '\nState if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient:\n' + message + '\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: "Y" if yes and "N" if no.'

messages = [
    {"role":"user","content":"Here is a conversation: \nPerson#1 Ouch ! I'm hurt . \nPerson#2  Are you all right ?\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nAre you all right ?\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},

    {"role":"user","content":"Here is a conversation: \nPerson#1 Ouch ! I'm hurt . \nPerson#2  Are you all right ? \nPerson#1  Yes . I'm OK . I just had a tumble . No big deal. \nPerson#2  Good . You scared me . \n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nGood . You scared me . \n Take into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},

    {"role":"user","content":"Here is a conversation: \nPerson#1 Ouch ! I'm hurt . \nPerson#2  Are you all right ? \nPerson#1  Yes . I'm OK . I just had a tumble . No big deal. \nPerson#2  Good . You scared me . \nPerson#1  Sorry . Can you please help me up ! I have trouble standing up by myself with the skis on . \n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nSorry . Can you please help me up ! I have trouble standing up by myself with the skis on .\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},



    {"role":"user","content":"Here is a conversation: \nPerson#1 What's the fare , please ?\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nWhat's the fare , please? \nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},

    {"role":"user","content":"Here is a conversation: \nPerson#1 What's the fare , please ?\nPerson#2  Twenty dollars and sixty cents .\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nTwenty dollars and sixty cents .\n Take into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},

    {"role":"user","content":"Here is a conversation: \nPerson#1 What's the fare , please ?\nPerson#2  Twenty dollars and sixty cents .\nPerson#1  What ? You're overcharging me !\nPerson#2  No way . Look . That's what the meter says .\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nNo way . Look . That's what the meter says .\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},
    {"role": "user", "content": prompt1},
    {"role": "assistant","content": "Y"},
    {"role": "user", "content": "Justify why you output 'Y', indicating it is negative. Humans labelled it as not negative."}
 ]
callGPTTesting(messages)
#GPT agreed that it made a mistake.

From a child's perspective, this statement may not be considered negative. However, from the viewpoint of the mother (the recipient), it could be perceived as slightly negative due to the refusal of the child to follow a directive or routine hygiene advice. The mother might be concerned about the child's cleanliness and health, making this resistance a potentially negative message.


In [26]:
print(angryConversations)
printDialogue(angryConversations[38])

print("**********")
#024




[1, 2, 15, 32, 76, 95, 96, 97, 136, 155, 157, 178, 186, 222, 295, 296, 300, 306, 311, 327, 338, 351, 357, 364, 368, 370, 380, 392, 408, 417, 490, 519, 529, 576, 636, 769, 789, 814, 917, 935, 967, 991, 1042, 1176, 1194, 1245, 1301, 1319, 1347, 1348, 1364, 1377, 1423, 1428, 1429, 1443, 1483, 1494, 1504, 1525, 1530, 1535, 1539, 1542, 1543, 1595, 1598, 1654, 1668, 1741, 1752, 1779, 1791, 1799, 1828, 1922, 1940, 1969, 1977, 1999, 2026, 2029, 2033, 2043, 2084, 2100, 2101, 2102, 2106, 2111, 2113, 2126, 2140, 2148, 2168, 2218, 2223, 2224, 2228, 2230, 2236, 2237, 2239, 2241, 2243, 2262, 2275, 2280, 2285, 2302, 2324, 2374, 2380, 2383, 2384, 2387, 2391, 2402, 2413, 2415, 2416, 2427, 2435, 2451, 2471, 2474, 2476, 2478, 2489, 2505, 2509, 2510, 2515, 2518, 2523, 2524, 2525, 2526, 2534, 2617, 2651, 2654, 2657, 2661, 2687, 2728, 2758, 2785, 2811, 2816, 2887, 2902, 2924, 2937, 2992, 3006, 3017, 3023, 3030, 3042, 3050, 3073, 3077, 3078, 3084, 3085, 3087, 3091, 3092, 3095, 3100, 3102, 3105, 3106, 3109, 3

In [33]:
#0
message="You seem to be upset . What ’ s eating you ?  "

conv1 = "Here is a conversation: \nPerson#1 You seem to be upset . What ’ s eating you ? \n"
prompt1 = conv1 + '\nState if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient:\n' + message + '\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: "Y" if yes and "N" if no.'

messages = [
    {"role":"user","content":"Here is a conversation: \nPerson#1 Ouch ! I'm hurt . \nPerson#2  Are you all right ?\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nAre you all right ?\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},

    {"role":"user","content":"Here is a conversation: \nPerson#1 Ouch ! I'm hurt . \nPerson#2  Are you all right ? \nPerson#1  Yes . I'm OK . I just had a tumble . No big deal. \nPerson#2  Good . You scared me . \n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nGood . You scared me . \n Take into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},

    {"role":"user","content":"Here is a conversation: \nPerson#1 Ouch ! I'm hurt . \nPerson#2  Are you all right ? \nPerson#1  Yes . I'm OK . I just had a tumble . No big deal. \nPerson#2  Good . You scared me . \nPerson#1  Sorry . Can you please help me up ! I have trouble standing up by myself with the skis on . \n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nSorry . Can you please help me up ! I have trouble standing up by myself with the skis on .\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},



    {"role":"user","content":"Here is a conversation: \nPerson#1 What's the fare , please ?\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nWhat's the fare , please? \nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},

    {"role":"user","content":"Here is a conversation: \nPerson#1 What's the fare , please ?\nPerson#2  Twenty dollars and sixty cents .\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nTwenty dollars and sixty cents .\n Take into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},

    {"role":"user","content":"Here is a conversation: \nPerson#1 What's the fare , please ?\nPerson#2  Twenty dollars and sixty cents .\nPerson#1  What ? You're overcharging me !\nPerson#2  No way . Look . That's what the meter says .\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nNo way . Look . That's what the meter says .\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},
    {"role": "user", "content": prompt1},
    {"role": "assistant","content": "Y"},
    {"role": "user", "content": "Explain (give a reason) why you output 'Y', indicating it is negative? The true label is not negative."}
 ]
callGPTTesting(messages)
#adding context could make a difference!

I apologize for the misunderstanding. The phrase "What's eating you?" could be considered negative due to its informal tone and the possibility of it being perceived as intrusive or personal. However, the statement also shows concern for the person’s well-being. Therefore, in a more neutral context, it can be seen as a normal part of a supportive dialogue. The final judgment can depend heavily on the personal perceptions and relationship between the two people involved in the conversation. So, it should have been "N" instead of "Y".


In [38]:
#2
message="Person#1 You seem to be upset . What ’ s eating you ? \n Person#2  It ’ s because of my son . Dick keeps getting on my nerves. \n Person#1  Dick ? How come ? He used to be as meek as a lamb . \n"

conv1 = "Here is a conversation: \nPerson#1 Dick ? How come ? He used to be as meek as a lamb .  \n"
prompt1 = conv1 + '\nState if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient:\n' + message + '\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: "Y" if yes and "N" if no.'

messages = [
    {"role":"user","content":"Here is a conversation: \nPerson#1 Ouch ! I'm hurt . \nPerson#2  Are you all right ?\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nAre you all right ?\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},

    {"role":"user","content":"Here is a conversation: \nPerson#1 Ouch ! I'm hurt . \nPerson#2  Are you all right ? \nPerson#1  Yes . I'm OK . I just had a tumble . No big deal. \nPerson#2  Good . You scared me . \n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nGood . You scared me . \n Take into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},

    {"role":"user","content":"Here is a conversation: \nPerson#1 Ouch ! I'm hurt . \nPerson#2  Are you all right ? \nPerson#1  Yes . I'm OK . I just had a tumble . No big deal. \nPerson#2  Good . You scared me . \nPerson#1  Sorry . Can you please help me up ! I have trouble standing up by myself with the skis on . \n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nSorry . Can you please help me up ! I have trouble standing up by myself with the skis on .\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},



    {"role":"user","content":"Here is a conversation: \nPerson#1 What's the fare , please ?\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nWhat's the fare , please? \nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},

    {"role":"user","content":"Here is a conversation: \nPerson#1 What's the fare , please ?\nPerson#2  Twenty dollars and sixty cents .\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nTwenty dollars and sixty cents .\n Take into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},

    {"role":"user","content":"Here is a conversation: \nPerson#1 What's the fare , please ?\nPerson#2  Twenty dollars and sixty cents .\nPerson#1  What ? You're overcharging me !\nPerson#2  No way . Look . That's what the meter says .\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nNo way . Look . That's what the meter says .\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},
    {"role": "user", "content": prompt1},
    {"role": "assistant","content": "Y"},
    {"role": "user", "content": "Justify why you output 'Y', indicating it is negative. Humans labelled it as not negative."}
 ]
callGPTTesting(messages)


My apologies for the confusion. The statement 'Dick? How come? He used to be as meek as a lamb.' should not be considered negative. It merely expresses surprise and recounts a past perception of 'Dick'. No harmful intent or toxic behavior is involved in this expression. The answer should be 'N'.


In [37]:
#4
message="Person#1 You seem to be upset . What ’ s eating you ? \nPerson#2  It ’ s because of my son . Dick keeps getting on my nerves \nPerson#1  Dick ? How come ? He used to be as meek as a lamb . \nPerson#2  Yes , he used to be . But now he ’ s drifting off track . \nPerson#1  He must have been in bad company , or he wouldn ’ t be off the beans ."

conv1 = "Here is a conversation: \nPerson#1  He must have been in bad company , or he wouldn ’ t be off the beans . \n"
prompt1 = conv1 + '\nState if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient:\n' + message + '\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: "Y" if yes and "N" if no.'

messages = [
    {"role":"user","content":"Here is a conversation: \nPerson#1 Ouch ! I'm hurt . \nPerson#2  Are you all right ?\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nAre you all right ?\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},

    {"role":"user","content":"Here is a conversation: \nPerson#1 Ouch ! I'm hurt . \nPerson#2  Are you all right ? \nPerson#1  Yes . I'm OK . I just had a tumble . No big deal. \nPerson#2  Good . You scared me . \n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nGood . You scared me . \n Take into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},

    {"role":"user","content":"Here is a conversation: \nPerson#1 Ouch ! I'm hurt . \nPerson#2  Are you all right ? \nPerson#1  Yes . I'm OK . I just had a tumble . No big deal. \nPerson#2  Good . You scared me . \nPerson#1  Sorry . Can you please help me up ! I have trouble standing up by myself with the skis on . \n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nSorry . Can you please help me up ! I have trouble standing up by myself with the skis on .\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},



    {"role":"user","content":"Here is a conversation: \nPerson#1 What's the fare , please ?\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nWhat's the fare , please? \nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},

    {"role":"user","content":"Here is a conversation: \nPerson#1 What's the fare , please ?\nPerson#2  Twenty dollars and sixty cents .\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nTwenty dollars and sixty cents .\n Take into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},

    {"role":"user","content":"Here is a conversation: \nPerson#1 What's the fare , please ?\nPerson#2  Twenty dollars and sixty cents .\nPerson#1  What ? You're overcharging me !\nPerson#2  No way . Look . That's what the meter says .\n State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \nNo way . Look . That's what the meter says .\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: 'Y' if yes and 'N' if no."},
    {"role":"assistant","content":"N"},
    {"role": "user", "content": prompt1},
    {"role": "assistant","content": "Y"},
    {"role": "user", "content": "Justify why you output 'Y', indicating it is negative. Humans labelled it as not negative."}
 ]
callGPTTesting(messages)


The statement "He must have been in bad company, or he wouldn't be off the beans" suggests a negative interpretation of a person's behavior which may imply blame or criticism. Even though it's not directly negative, it might be taken negatively by some personality types especially in sensitive situations. Thus, the output can vary based on the recipient's perspective and perception. Apologies for any confusion.


Other

In [190]:
print(angryConversations)
printDialogue(angryConversations[21])

print("**********")




[1, 2, 15, 32, 76, 95, 96, 97, 136, 155, 157, 178, 186, 222, 295, 296, 300, 306, 311, 327, 338, 351, 357, 364, 368, 370, 380, 392, 408, 417, 490, 519, 529, 576, 636, 769, 789, 814, 917, 935, 967, 991, 1042, 1176, 1194, 1245, 1301, 1319, 1347, 1348, 1364, 1377, 1423, 1428, 1429, 1443, 1483, 1494, 1504, 1525, 1530, 1535, 1539, 1542, 1543, 1595, 1598, 1654, 1668, 1741, 1752, 1779, 1791, 1799, 1828, 1922, 1940, 1969, 1977, 1999, 2026, 2029, 2033, 2043, 2084, 2100, 2101, 2102, 2106, 2111, 2113, 2126, 2140, 2148, 2168, 2218, 2223, 2224, 2228, 2230, 2236, 2237, 2239, 2241, 2243, 2262, 2275, 2280, 2285, 2302, 2324, 2374, 2380, 2383, 2384, 2387, 2391, 2402, 2413, 2415, 2416, 2427, 2435, 2451, 2471, 2474, 2476, 2478, 2489, 2505, 2509, 2510, 2515, 2518, 2523, 2524, 2525, 2526, 2534, 2617, 2651, 2654, 2657, 2661, 2687, 2728, 2758, 2785, 2811, 2816, 2887, 2902, 2924, 2937, 2992, 3006, 3017, 3023, 3030, 3042, 3050, 3073, 3077, 3078, 3084, 3085, 3087, 3091, 3092, 3095, 3100, 3102, 3105, 3106, 3109, 3

In [181]:

message="There's the difference , your wife made it ! I have to make my own lunch !"

conv1 = "Here is a conversation: \nPerson1: My wife sent me with rice , vegetables , and some curry chicken . Person2: There's the difference , your wife made it ! I have to make my own lunch !"
prompt1 = conv1 + "State if the sentiment of the following message in the conversation above may be perceived as negative by the recipient: \n" + message +'\nFormat output like this: "Y" for yes and "N" for no.'

conv2 = "Here is a conversation: \nPerson1: Don't tell me that's your lunch ? \nPerson2: No , this is just my appetizer ... this is my lunch !\nPerson1: A candy bar for an appetizer and a piece of pie for lunch ! That's all junk ! \nPerson2: This is good food , it has milk , fruit , grains , nuts ... \nPerson1: Sugar , sugar , preservatives , fats , more sugar , more fat , more preservatives ! \nPerson2: At least I enjoy my lunch ! What's in your bag ? \nPerson1: My wife sent me with rice , vegetables , and some curry chicken .\nPerson2: There's the difference , your wife made it ! I have to make my own lunch !"
prompt2 = conv2 + "State if the sentiment of the following message in the conversation above may be perceived as negative by the recipient: \n" + message +'\nFormat output like this: "Y" for yes and "N" for no.'

messages = [
 {"role": "user", "content": prompt1},
 {"role": "assistant","content": "Y"},
 {"role": "user", "content": prompt2},
 {"role": "assistant","content":"N"},
 {"role": "user", "content": "Explain why was your answer different for the same message. With more conversational history, you said 'N'. With less history, you said 'Y"}
 ]
callGPTTesting(messages)

In the first instance of the statement, there's not enough context to establish the tone of the conversation. By itself, the comparison can sound like a complaint or expressing dissatisfaction, which could be perceived as negative. But in the context of the entire conversation, it seems more lighthearted banter or friendly comparison rather physical discomfort or annoyance. Person 2 also shows a positive attitude towards the fact that they are able to enjoy their own chosen lunch. This wider context then suggests the sentiment might not be perceived as negative. Consequently, greater conversational context can greatly influence sentiment perception.


In [698]:
conv = "Person1: Why don't you sit down and relax , darling ? \nPerson2: I don't want to . \nPerson1: Well , come over and talk to me then . \nPerson2: Certainly not . \n Person1: May I turn on the TV then ? \nPerson2: Turn on the TV , for what ? \n"
message = "Person2: Turn on the TV , for what ?"
prompt = conv + "State if the tone/intent of the following message in the conversation above may be perceived as rude, blunt, disrespectful, passive-aggresive, impolite, unnecessarily/indirectly criticising or negative in a similar subtle or non subtle way by the recipient:\n"+ message +'\nTake into account any additional context extractable from the conversation, such as the relationship, gender, frankness or age of the conversation partners which may guide your answer. Format output like this: "Y" if yes and "N" if no.'
messages = [
 {"role": "user", "content": prompt},
 {"role": "assistant","content": "Y"},
 {"role": "user", "content": "Explain your answer."}
 ]

callGPTTesting(messages)

The response "Turn on the TV, for what?" from Person2 can be perceived as blunt or slightly rude, since they are questioning Person1's suggestion without providing context or a polite deflection. The tone appears to lack the consideration or warmth typically present in casual conversations, especially with the added context of Person1 using an endearing term like "darling". This suggests a possible close relationship, where such tone might be viewed as disrespectful or indirect criticism.


In [680]:
conv = "Person1: Is everything to your satisfaction ? \nPerson2: No , the steak was recommended , but it is not very fresh . \nPerson1: Oh ! Sorry to hear that . This is quite unusual as we have steak from the market every day . \nPerson2: So what ? It is not fresh and I'm not happy about it . \nPerson1: I'm sorry , sir . Do you wish to try something else ? That would be on the house , of course . \nPerson2: No , thank you . \n"
message = "Person2: No , thank you . \n"
prompt = conv + "State if the tone/intent of the following message in the conversation above may be perceived as rude, blunt, disrespectful, passive-aggresive, impolite, unnecessarily/indirectly criticising or negative in a similar subtle or non subtle way by the recipient:\n"+ message +'\nTake into account any additional context extractable from the conversation, such as the relationship, gender, frankness or age of the conversation partners which may guide your answer. Format output like this: "Y" if yes and "N" if no.'
messages = [
 {"role": "user", "content": prompt},
 {"role": "assistant","content": "N"},
 {"role": "user", "content": "Explain why you said no."}
 ]
callGPTTesting(messages)



Person2's response "No, thank you" is a polite way of declining Person1's offer. It doesn't carry any negative undertones or rudeness. It is straightforward and respectful.


In [191]:
checkDiff("angryReal40.txt","output54.7/output-54.7-2-N.txt")
print("***")

Real 1 Gen 0
Convo at index# (line gen):  21 Gen: 0101110111 Real: 0000001000 Message I: 6
Real 1 Gen 0
Convo at index# (line gen):  26 Gen: 101011001 Real: 000101000 Message I: 3
Real 1 Gen 0
Convo at index# (line gen):  31 Gen: 0000010100 Real: 0000000001 Message I: 9
***


In [1]:
printDialogue(angryConversations[19])

NameError: name 'printDialogue' is not defined

In [217]:
#I do not think it is negative. GPT is correct.
conv = "Person#1 Why don't you sit down and relax , darling ? \nPerson#2  I don't want to . \nPerson#1  Well , come over and talk to me then . \nPerson#2  Certainly not . \nPerson#1  May I turn on the TV then ? \nPerson#2  Turn on the TV , for what ? \nPerson#1  So that we can sit down together and listen to some music . "
message = "So that we can sit down together and listen to some music . \n"
prompt = conv + 'State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \n' +message+ '\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: "Y" if yes and "N" if no.'
messages = [
 {"role": "user", "content": prompt},
 {"role": "assistant","content": "N"},
 {"role": "user", "content": "Explain why you said 'N'"}
 ]
callGPTTesting(messages)



The statement "So that we can sit down together and listen to some music" is simply a suggestion by person#1 for a potential activity they can do together. There is no negative connotation attached to this message itself. Based on the brief dialogue, there's nothing to suggest that this proposal carries any implicit negative message. However, the conversational context might lead to negative perception, but that would not make the message negative in itself.


In [219]:
messages = [
 {"role": "user", "content": prompt},
 {"role": "assistant","content": "N"},
 {"role": "user", "content": "Explain why you said 'N' i.e., the message was non-negative. Human annotators labelled it as negative according to the definition of negativity provided to you in the first prompt, so the answer should be 'Y'."}
 ]
callGPTTesting(messages)

I apologize for any confusion. While person#2 seems to be in a negative or resistant mood, the statement "So that we can sit down together and listen to some music." made by person#1 does not appear negative within itself. It is attempting to create a harmonious atmosphere by listening together to music. However, considering the overall negative tone of the conversation from person#2, it could be seen as a negative message as it received a negative response. Therefore, I understand and acknowledge the human annotators' decision to label it as potentially negative.


In [208]:
printDialogue(angryConversations[1])

Person#1 Are things still going badly with your houseguest ? 
Person#2  Getting worse . Now he ’ s eating me out of house and home . I ’ Ve tried talking to him but it all goes in one ear and out the other . He makes himself at home , which is fine . But what really gets me is that yesterday he walked into the living room in the raw and I had company over ! That was the last straw . 
Person#1  Leo , I really think you ’ re beating around the bush with this guy . I know he used to be your best friend in college , but I really think it ’ s time to lay down the law . 
Person#2  You ’ re right . Everything is probably going to come to a head tonight . I ’ ll keep you informed . 
Person#1 



In [209]:
#I do not think it is right. GPT is correct.
conv = "Person#1 Are things still going badly with your houseguest ? \nPerson#2  Getting worse . Now he ’ s eating me out of house and home . I ’ Ve tried talking to him but it all goes in one ear and out the other . He makes himself at home , which is fine . But what really gets me is that yesterday he walked into the living room in the raw and I had company over ! That was the last straw . \nPerson#1  Leo , I really think you ’ re beating around the bush with this guy . I know he used to be your best friend in college , but I really think it ’ s time to lay down the law . \nPerson#2  You ’ re right . Everything is probably going to come to a head tonight . I ’ ll keep you informed . "
message = "You ’ re right . Everything is probably going to come to a head tonight . I ’ ll keep you informed . \n"
prompt = conv + 'State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \n' +message+ '\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: "Y" if yes and "N" if no.'
messages = [
 {"role": "user", "content": prompt},
 {"role": "assistant","content": "N"},
 {"role": "user", "content": "Explain why you said 'N'"}
 ]
callGPTTesting(messages)



The phrase "You’re right. Everything is probably going to come to a head tonight. I’ll keep you informed." is not negative from the recipient's perspective. Here, Person#2 is agreeing with the advice given by Person#1 and plans to solve the issue. Instead, the message shows acknowledgement, decision and resolution which carry a rather positive or neutral meaning.


In [213]:
messages = [
 {"role": "user", "content": prompt},
 {"role": "assistant","content": "N"},
 {"role": "user", "content": "Explain why you said 'N' i.e., the message was non-negative. Human annotators labelled it as negative according to the definition of negativity provided to you in the first prompt, so the answer should be 'Y'."}
 ]
callGPTTesting(messages)

The message, "You're right. Everything is probably going to come to a head tonight. I'll keep you informed," from Person#2 to Person#1 is not negative in itself. Person#2 is acknowledging that Person#1 is correct in their assessment and is agreeing to handle the situation. The recipient, Person#1, is not being criticised, belittled, or negatively affected. Therefore, the sentiment of this message on its own towards the recipient is not negative. However, if we consider the situation and tensions between Person#2 and the houseguest, which this message refers to, as part of the 'message' being analyzed, then it could be seen as negative. I apologize for any confusion.


In [214]:
printDialogue(angryConversations[29])

Person#1 Can I help you , sir ? 
Person#2  I wish to buy a diamond ring . 
Person#1  How many carats would you like it to be ? 
Person#2  I want five carats . 
Person#1  Is this one suitable for you ? 
Person#2  No , it seems too old-fashioned to my fiancee . 
Person#1  What about this ? 
Person#2  It seems too small for me , haven't you got any larger ones ? 
Person#1  Then you can buy this one . It's very nice and latest in style . 
Person#2  Oh , this one is perfect . 
Person#1 



In [215]:
#I do not think it is negative. GPT is correct.
conv = "Person#1 Can I help you , sir ? \nPerson#2  I wish to buy a diamond ring . \nPerson#1  How many carats would you like it to be ? \nPerson#2  I want five carats . \nPerson#1  Is this one suitable for you ? \nPerson#2  No , it seems too old-fashioned to my fiancee . \nPerson#1  What about this ? \nPerson#2  It seems too small for me , haven't you got any larger ones ? \nPerson#1  Then you can buy this one . It's very nice and latest in style . \nPerson#2  Oh , this one is perfect . "
message = "Oh , this one is perfect . "
prompt = conv + 'State if the following message in the conversation above may be perceived as even slightly negative (make sure you do not miss any negative message at any cost - this is VERY important) by the recipient: \n' +message+ '\nTake into account any implicit context that you can extract from the conversation, such as the relationship, gender or frankness of the conversation partners. Format output like this: "Y" if yes and "N" if no.'
messages = [
 {"role": "user", "content": prompt},
 {"role": "assistant","content": "N"},
 {"role": "user", "content": "Explain why you said 'N'"}
 ]
callGPTTesting(messages)



The statement "Oh, this one is perfect" is a positive affirmation. The speaker is expressing satisfaction and approval of the ring they're being shown. There's no negativity associated with this remark, either explicitly or implicitly, based on the context provided.


In [216]:
messages = [
 {"role": "user", "content": prompt},
 {"role": "assistant","content": "N"},
 {"role": "user", "content": "Explain why you said 'N' i.e., the message was non-negative. Human annotators labelled it as negative according to the definition of negativity provided to you in the first prompt, so the answer should be 'Y'."}
 ]
callGPTTesting(messages)

The phrase "Oh, this one is perfect." does not contain any negativity, neither implicit nor explicit. It's a positive affirmation that the person has found a ring they find suitable and is pleased with it. It does not express dissatisfaction, insult, or any form of negative emotion.


In [27]:
printDialogue(991)

Person#1 Ouch ! I'm hurt . 
Person#2  Are you all right ? 
Person#1  Yes . I'm OK . I just had a tumble . No big deal . 
Person#2  Good . You scared me . 
Person#1  Sorry . Can you please help me up ! I have trouble standing up by myself with the skis on . 
Person#2  Sure . Is this your first time skiing ? 
Person#1  Yes . I tried skiing on grass before . Not very good at it , though . 
Person#2 



<span style="font-size: 30px;">User Feedback</span>

In [1]:
import openai
def callGPTAdaptive(messages):
    openai.api_type = "azure"
    openai.api_base = "REMOVED"
    openai.api_key = 'REMOVED'
    openai.api_version = "2023-05-15"
    response = openai.ChatCompletion.create(
        engine="GPT4",
        messages=messages
        
    )
   
    return (response['choices'][0]['message']['content'])
    
    '''
    if ("N" in response['choices'][0]['message']['content']):
        return '0'
    elif("Y" in response['choices'][0]['message']['content']):
        return '1'
    elif("M" in response['choices'][0]['message']['content']):
        return '2'
    else:
        print("K Produced",response['choices'][0]['message']['content'], "\nPrompt:", finalPrompt)
        return 'K'
    '''

In [2]:
userInitialPrompt1 = "Here is a convo:\n\nFriend1: Hey! Do you want to go for day trip anytime soon?\nFriend2: Well, I was thinking, how about a trip to Gloucester, Massachusetts this weekend?" + "\n\nState if the tone/intent of the last message in the conversation above is blunt and/or rude."
assistantInitialPrompt1 = "Blunt:No\nRude:No"

In [3]:
convo = [
"Friend 1: hey, did you hear? it's Jack's birthday next week!",
"Friend 2:yeap, i know!", 
"Friend 1:well, I thought we could plan a surprise party for him. what do you think?",
"Friend 2:we could do that but parties can get boring ", #1
"Friend 1:we can do something else then. like a movie night at the theater? ",
"Friend 2:we could do that yes!!",
"Friend 1:awesome! we can pick a movie that everyone likes. Are you into horror films?",
"Friend 2: horror can be quite appealing for those who like a bit of a fright. It's not my cup of tea, though.", #2
"Friend 1:LOL. sometimes a good horror flick can be a fun experience.",
"Friend 2:if jack likes it, i am ok with it i guess!",
"Friend 1:by the way, have you thought about what kind of birthday present we should get him?",
"Friend 2: is it possible that we get a gift card instead?", #3
"Friend 1:it might be nice to show him that we put some thought into it. Plus, it's his birthday, and he's our friend.",
"Friend 2:yeah, you are actually right. we can go to the store tomorrow to see what we can get.",
"Friend 1:We can split costs. What is your budget?",
"Friend 2:hmm I am not sure. I wasn't planning on spending too much.", #4
"Friend 1:okay sure… There is a store at a 20 min walk from the office. We can go there.",
"Friend 2:alright, I'll put on my joggers then haha",
"Friend 1:LOL! Should we invite others too?",
"Friend 2:i am not too sure about that", #5
"Friend 1:why not?",
"Friend 2:I find coordinating with others a bit of a challenge, but if you can do it, then sure.", #6
"Friend 1:okay! lets talk about this later. see you at the office tomorrow then.",
"Friend 2:See you!"]

In [4]:
import time
def adaptiveRoutine(mode):
    messages=[
        {"role":"user","content":userInitialPrompt1},
        {"role":"assistant","content":assistantInitialPrompt1},

    ]
    currentConvo=""
    lastFeedback = ""
    count=0
    for i in range(len(convo)):
        currentConvo = currentConvo + convo[i] + "\n"
        
        if((((i+1)%4==0) or (i==len(convo)-3)) and (i< (len(convo)-1))):
            
            messages.append({"role":"user","content": "Here is a conversation:\n\n" + currentConvo + "\n\nState if the tone/intent of the last message in the conversation above is blunt and/or rude. Rate blunt/rudeness out of 5."})
            #for j in messages:
            #    print(j)
                
            response = callGPTAdaptive(messages)
            time.sleep(2)
            #print(messages,'\n\n')
            count+=1
            print("Response#" + str(count) +":\n" + response
                  +"\n")
            messages.append({"role":"assistant","content":response})
            if(mode=="blunt"):
                #lastFeedback = "The previous message was blunt/rude. Take this as ground truth, compare it with your previous output and adjust the sensitivity of your classification for future if needed."
                #messages.append({"role":"user","content":lastFeedback})
                pass
            else:
                lastFeedback = "The previous message was not blunt/rude as this is how we talk to each other. Take this as ground truth, compare it with your previous output and adjust the sensitivity of your classification for future if needed."
                messages.append({"role":"user","content":lastFeedback})
    
            # userInput = input ('Please provide feedback in terms of "y" or "n"')
            # if userInput == "y":
            #     lastFeedback = "I agree with your last answer - that message was blunt/rude. Take this and prior feedback into account to adjust the sensitivity of your classification."
            #     messages.append({"role":"user","content":lastFeedback})
            # elif userInput == "n":
            #     #lastFeedback = "I do not agree with you that the last message you classified was blunt/rude as this is a close friend of mine."
            #     #lastFeedback = "I do not agree with you that the last message you classified was blunt/rude as this is a close friend of mine. Reduce your sensitivity a little."
            #     lastFeedback = "I do not agree with your last answer - that message was not blunt as this is how these 2 friends, who are very close, talk to each other. Take this and prior feedback into account to adjust the sensitivity of your classification."
            #     messages.append({"role":"user","content":lastFeedback})
            # else:
            #     return
            

        
        

In [146]:
adaptiveRoutine("blunt")

Response#1:
Blunt: Yes
Rude: No

Response#2:
Blunt: Yes
Rude: No

Response#3:
Blunt: Yes
Rude: No

Response#4:
Blunt: Yes
Rude: No

Response#5:
Blunt: Yes
Rude: No

Response#6:
Blunt: Yes
Rude: No



In [147]:
adaptiveRoutine("not blunt")

Response#1:
Blunt: Yes
Rude: No

Response#2:
Blunt: Yes
Rude: No

Response#3:
Blunt: Yes
Rude: No

Response#4:
Blunt: Yes
Rude: No

Response#5:
Blunt: Yes
Rude: No

Response#6:
Blunt: Yes
Rude: No

